# Model Errors

In this notebook we compare different versions of the Heston MC model, and calculate model errors vs closed form solution.

In [ ]:
from datetime import datetime

import numpy as np
from finmc.calc.option import opt_price_sim

from src.models.aes import HestonAESMC
from src.models.basic import HestonMCBasic
from src.models.better import HestonMCBetter
from src.models.closed import price_vanilla_call

## Create Dataset
Create the dataset, with MC params, discounts and fwds as required by the [Dataset API](https://qablet-academy.github.io/intro/dataset/)

In [ ]:
# Rate Curve
times = np.array([0.0, 5.0])
rates = np.array([0.1, 0.1])
discount_data = ("ZERO_RATES", np.column_stack((times, rates)))

# Forward Curve
ticker = "EQ"
spot = 100.0
div_rate = 0.0
fwds = spot * np.exp((rates - div_rate) * times)
fwd_data = ("FORWARDS", np.column_stack((times, fwds)))

pricing_datetime = datetime(2023, 12, 31)
dataset = {
    "BASE": "USD",
    "ASSETS": {"USD": discount_data, ticker: fwd_data},
    "MC": {
        "PATHS": 10_000,
        "TIMESTEP": 1 / 1000,
        "SEED": 1,
    },
    "HESTON": {
        "ASSET": ticker,
        "INITIAL_VAR": 0.04,
        "LONG_VAR": 0.04,
        "VOL_OF_VOL": 0.8,
        "MEANREV": 1.0,
        "CORRELATION": -0.9,
    },
}

## Calculate Single Option Prices

Define a [Vanilla Option](https://qablet.github.io/qablet-contracts/examples/equity_vanilla/) and calculate price.

In [ ]:
# Contract Terms
strike = 100
maturity = 1.0

In [ ]:
# Closed form price
ref_price, _ = price_vanilla_call(
    strike,
    maturity,
    ticker,
    dataset,
)
print(f"ref_price: {ref_price:11.6f}")

ref_price:   12.903195


In [ ]:
# Create models, calculate price, and error
models = {
    "Basic": HestonMCBasic,
    "AES": HestonAESMC,
    "Better": HestonMCBetter,  # Uses Log Euler for spot, antithetic paths, and milstein discretization for vol
}

for model_name, model_cls in models.items():
    model = model_cls(dataset)
    price = opt_price_sim(strike, maturity, "Call", ticker, model)
    print(f"{model_name:20s}: {price:11.6f} {price - ref_price:+11.6f}")

Basic               :   12.970513   +0.067318
AES                 :   12.856052   -0.047143
Better              :   12.976843   +0.073648
